In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from astropy.io import fits
from astropy.table import Table
from astropy.coordinates import SkyCoord
from astropy import units as u

In [9]:
det_cats = {
    'pz14' : pd.read_csv('./ep14_thr0.1.csv'),
    'pz20' : pd.read_csv('./full_pz20_thr0.1_step8.csv'),
    'pz25' : pd.read_csv('./full_pz25_thr0.1_step8.csv'),
    'pz40' : pd.read_csv('./ep40_thr0.1.csv'),
    'pz_act10' : pd.read_csv('./full_act_cut_ep10_thr0.1_step8.csv'),
    'pz_act14' : pd.read_csv('./full_act_cut_ep14_thr0.1_step8.csv'),
    'pz_act20' : pd.read_csv('./full_pz_act20_thr0.1_step8.csv'),
    'pz_act25' : pd.read_csv('./full_pz_act25_thr0.1_step8.csv')
}

In [5]:
erosita = pd.read_csv('./erosita.csv')

In [8]:
match_dist = 5 / 60

In [ ]:
def calc_error(det_cat, true_cat, shift=15/60, match_dist=5/60, n_try=10, seed=0):
    import numpy as np
    from astropy.coordinates import SkyCoord
    from astropy import units as u
    
    error = []
    np.random.seed(seed)
    for i in range(n_try):
        det_sc = SkyCoord(ra=np.array(det_cat['RA']) * u.degree, 
                          dec=np.array(det_cat['DEC']) * u.degree, frame='icrs')
        angles = np.random.randint(0, 360, len(det_cat))
        det_sc = det_sc.directional_offset_by(angles*u.degree, shift)

        true_sc = SkyCoord(ra=np.array(true_cat['RA']) * u.degree, 
                           dec=np.array(true_cat['DEC']) * u.degree, frame='icrs')
        _, d2d, _ = det_sc.match_to_catalog_sky(true_sc)
        c_error = np.count_nonzero(d2d.degree < match_dist)
        error.append(c_error)
    return np.array(error).mean()

In [31]:
comp_df = []
recall_df = []
for det_name in det_cats:
    line = {}
    line_r = {}
    
    sc = SkyCoord(ra=np.array(det_cats[det_name]['RA'])*u.degree, 
                  dec=np.array(det_cats[det_name]['DEC'])*u.degree, frame='icrs')
    
    tr_sc = SkyCoord(ra=np.array(erosita['RA'])*u.degree, 
                  dec=np.array(erosita['DEC'])*u.degree, frame='icrs')
    idx, d2d, _ = sc.match_to_catalog_sky(tr_sc)
    matched = d2d.degree <= match_dist
    line['erosita'] = np.count_nonzero(det_cats[det_name].iloc[matched]['status'] != 'fn')
    line['erosita'+'_err'] = calc_error(det_cats[det_name], erosita)
    line_r['erosita'] = line['erosita'] / len(erosita)
    
    line['fp'] = np.count_nonzero(det_cats[det_name]['status'] == 'fp')
    line_r['fp'] = line['fp']
    comp_df.append(pd.DataFrame(line, index=[det_name]))
    recall_df.append(pd.DataFrame(line_r, index=[det_name]))

line = {}
line['erosita'] = len(erosita)
line['erosita'+'_err'] = 0
line['fp'] = 0
comp_df.append(pd.DataFrame(line, index=['all']))
comp_df = pd.concat(comp_df)
recall_df = pd.concat(recall_df)

In [32]:
comp_df

,psz2,psz2_err,mcxc,mcxc_err,rm,rm_err,act,act_err,fp
pz14,1491,22.4,725,23.1,1242,399.4,849,70.5,15828
pz20,1528,23.6,740,25.5,1306,406.6,875,72.2,23104
pz25,1525,25.5,747,27.3,1374,448.2,888,75.2,20611
pz40,1506,22.5,739,23.5,1279,427.8,871,72.4,17306
pz_act10,1361,17.3,659,18.8,1029,283.9,926,46.6,16316
pz_act14,1363,18.0,670,19.0,1211,334.2,1282,57.2,16484
pz_act20,1217,15.4,599,15.9,1182,261.2,1866,47.8,9398
pz_act25,1260,19.6,610,17.4,1271,287.8,2152,58.0,15275
all,1653,0.0,1743,0.0,26111,0.0,4195,0.0,0


In [18]:
recall_df

,psz2,mcxc,rm,act,fp
pz14,0.901996,0.415950,0.047566,0.202384,15828
pz20,0.924380,0.424555,0.050017,0.208582,23104
pz25,0.922565,0.428571,0.052622,0.211681,20611
pz40,0.911071,0.423982,0.048983,0.207628,17306
pz_act10,0.823351,0.378084,0.039409,0.220739,16316
pz_act14,0.824561,0.384395,0.046379,0.305602,16484
pz_act20,0.736237,0.343660,0.045268,0.444815,9398
pz_act25,0.762250,0.349971,0.048677,0.512992,15275
